In [56]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import Subset
import torch.optim as optim
import torch.nn.functional as F

In [57]:
# === CONFIG ===
INPUT_DIR = "/home/HardDisk/Satang/thesis_proj/New_45/15/split_tws/X_csv_split_31"
CHUNK_SIZE = 31
NUM_FEATURES = 8
NUM_EPOCHS = 50
BATCH_SIZE = 32
K_FOLDS = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [58]:
# === HELPERS ===
def collate_fn(batch):
    streams, labels = zip(*batch)
    streams = list(zip(*streams))
    streams = [torch.stack(s) for s in streams]
    labels = torch.tensor(labels)
    return streams, labels

def compute_class_weights(labels, device):
    counter = Counter(labels)
    total = sum(counter.values())
    weights = [np.log(total / (counter[i] + 1)) for i in range(len(counter))]
    return torch.tensor(weights, dtype=torch.float).to(device)

def print_model_info(model):
    print("Model Architecture:")
    print(model)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")

def plot_loss_curve(train_losses, val_losses):
    epochs = range(len(train_losses))
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_losses, label='Train Loss', color='blue')
    plt.plot(epochs, val_losses, label='Val Loss', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_accuracy_curve(train_acc, val_acc):
    epochs = range(len(train_acc))
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_acc, label='Train Accuracy', color='green')
    plt.plot(epochs, val_acc, label='Val Accuracy', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [59]:
# === DATASET CLASS ===
class MultiStreamDataset(Dataset):
    def __init__(self, data, labels, label_encoder, augment=False):
        self.data = data
        self.labels = label_encoder.transform(labels)
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def augment_stream(self, stream):
        jitter = np.random.normal(0, 0.01, stream.shape)
        scale = np.random.normal(1.0, 0.05, stream.shape)
        return stream * scale + jitter

    def __getitem__(self, idx):
        sample = self.data[idx]  # shape: (T, 8)
        
        if self.augment:
            # Apply augmentation to each feature independently
            jitter = np.random.normal(0, 0.01, sample.shape)
            scale = np.random.normal(1.0, 0.05, sample.shape)
            sample = sample * scale + jitter

        sample_tensor = torch.tensor(sample, dtype=torch.float32)  # shape: (T, 8)
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.long)
    
        return sample_tensor, label_tensor


In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class AEWithClassifier(nn.Module):
    def __init__(self, input_length=31, feature_dim=8, latent_dim=64, num_classes=12, proj_dim=128):
        super(AEWithClassifier, self).__init__()

        self.encoder_out_len = math.ceil(input_length / 2)  # ceil(31/2) = 16

        # === Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(feature_dim, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * self.encoder_out_len, latent_dim),
            nn.ReLU()
        )

        # === Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64 * self.encoder_out_len),
            nn.ReLU(),
            nn.Unflatten(1, (64, self.encoder_out_len)),
            nn.ConvTranspose1d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.Conv1d(32, feature_dim, kernel_size=3, padding=1)
        )

        # === Classifier head
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

        # === Projection for KD (feature distillation)
        self.projection = nn.Linear(latent_dim, proj_dim)

    def forward(self, x, return_features=False):
        """
        return_features:
            - False → returns logits and recon (for training)
            - True  → returns logits, recon, and projected feature (for KD)
        """
        T_orig = x.shape[1]               # (B, 31, 8)
        x = x.permute(0, 2, 1)            # → (B, 8, 31)
        latent = self.encoder(x)          # → (B, latent_dim)
        recon = self.decoder(latent).permute(0, 2, 1)
        recon = recon[:, :T_orig, :]      # trim to match input shape
        logits = self.classifier(latent)
        proj_feat = self.projection(latent)  # → (B, proj_dim)

        if return_features:
            return logits, recon, proj_feat
        else:
            return logits, recon


In [61]:

# # === COLLATE FUNCTION ===
# def collate_fn(batch):
#     streams, labels = zip(*batch)
#     streams = list(zip(*streams))
#     streams = [torch.stack(s) for s in streams]
#     labels = torch.tensor(labels)
#     return streams, labels

# === LOAD SPLIT FUNCTION ===
def load_split_from_folder(split_dir, expected_shape):
    X, y = [], []
    for class_name in sorted(os.listdir(split_dir)):
        class_path = os.path.join(split_dir, class_name)
        for fname in sorted(os.listdir(class_path)):
            if fname.endswith(".csv"):
                fpath = os.path.join(class_path, fname)
                chunk = pd.read_csv(fpath, header=None).values
                if chunk.shape == expected_shape:
                    X.append(chunk)
                    y.append(class_name)
    return np.array(X), np.array(y)

In [62]:
# === APPLY SMOTE ===
def apply_smote_on_training(X_chunks, y_labels, chunk_size, num_features):
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_labels)

    smote_encoder = LabelEncoder()
    y_encoded_for_smote = smote_encoder.fit_transform(y_labels)

    X_flat = X_chunks.reshape(X_chunks.shape[0], -1)
    X_resampled, y_resampled = SMOTE().fit_resample(X_flat, y_encoded_for_smote)

    X_res = X_resampled.reshape(-1, chunk_size, num_features)
    y_res_str = smote_encoder.inverse_transform(y_resampled)

    return X_res, y_res_str, label_encoder

# # === LOAD DATASETS ===
# expected_shape = (CHUNK_SIZE, NUM_FEATURES)

# X_train_raw, y_train_raw = load_split_from_folder(os.path.join(INPUT_DIR, "train"), expected_shape)
# X_val_raw,   y_val_raw   = load_split_from_folder(os.path.join(INPUT_DIR, "val"), expected_shape)

# # === SMOTE ONLY ON TRAIN ===
# X_train_balanced, y_train_str, label_encoder = apply_smote_on_training(
#     X_train_raw, y_train_raw, CHUNK_SIZE, NUM_FEATURES
# )

# # === CREATE DATASETS ===
# train_dataset = MultiStreamDataset(X_train_balanced, y_train_str, label_encoder, augment=True)
# val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [63]:
# === CLASS WEIGHTING ===
def compute_class_weights(labels, device):
    from collections import Counter
    total = len(labels)
    counts = Counter(labels)
    weights = [np.log(total / (counts[i] + 1)) for i in range(len(counts))]
    return torch.tensor(weights, dtype=torch.float).to(device)

# class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device="cpu")

In [64]:
def train_model(model, train_loader, val_loader, device, epochs=50, lr=0.001,
                class_weights=None, optimizer=None, scheduler=None,
                alpha=1.0, beta=1.0, best_model_path="ae_classifier_model.pth"):

    print("🔍 Class Weights Tensor:")
    print(class_weights)

    criterion_ce = nn.CrossEntropyLoss(weight=class_weights)
    criterion_mse = nn.MSELoss()

    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    best_val_acc = 0

    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0.0, 0

        if scheduler:
            scheduler.step()

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits, recon = model(inputs)

            loss_ce = criterion_ce(logits, labels)
            loss_mse = criterion_mse(recon, inputs)
            loss = alpha * loss_ce + beta * loss_mse

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()

            total_loss += loss.item()
            correct += (logits.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)

        # === Validation ===
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logits, recon = model(inputs)

                loss_ce = criterion_ce(logits, labels)
                loss_mse = criterion_mse(recon, inputs)
                loss = alpha * loss_ce + beta * loss_mse

                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {total_loss:.4f} - Val Loss: {val_loss:.4f} - "
              f"Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"💾 Best model saved to: {best_model_path}")

    model.load_state_dict(torch.load(best_model_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [65]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torch_lr_finder import LRFinder
from imblearn.over_sampling import SMOTE

# === 🧩 Config ===
NUM_EPOCHS = 50
LATENT_DIM = 64
BEST_LR = 0.01
base_dir = "/home/HardDisk/Satang/thesis_proj"
save_root = "/home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher"
os.makedirs(save_root, exist_ok=True)

detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]

for T_d in detection_times:
    for T_w in window_sizes:
        print(f"\n🚀 Running for Td={T_d}, Tw={T_w}")
        model_name = os.path.join(save_root, f"ae_Td{T_d}_Tw{T_w}.pth")
        T_len = T_d - T_w + 1
        folder_name = f"X_csv_split_{T_len}"

        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")

        # === 1. Load Data ===
        expected_shape = (T_len, NUM_FEATURES)
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. SMOTE + Encode ===
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)
        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, expected_shape[0], NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Datasets + Loaders ===
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === 4. AE Model ===
        model = AEWithClassifier(
            input_length=expected_shape[0],
            feature_dim=NUM_FEATURES,
            latent_dim=LATENT_DIM,
            num_classes=len(label_encoder.classes_)
        ).to(device)

        optimizer = optim.AdamW(model.parameters(), lr=BEST_LR, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 5. Train AE ===
        train_accs, val_accs, train_losses, val_losses = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            class_weights=class_weights_tensor,
            lr=BEST_LR,
            optimizer=optimizer,
            scheduler=scheduler,
            best_model_path=model_name,
            alpha=1.0,  # CrossEntropy
            beta=1.0    # MSE Reconstruction
        )

        print(f"✅ Finished Td={T_d}, Tw={T_w} — saved to {model_name}")



🚀 Running for Td=30, Tw=10
🔍 Class Weights Tensor:
tensor([2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843,
        2.4843, 2.4843, 2.4843], device='cuda:0')


/home/ea301b/anaconda3/envs/dl_satang/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1/50 - Train Loss: 665.3254 - Val Loss: 61.3876 - Train Acc: 0.5477 - Val Acc: 0.5952
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td30_Tw10.pth
Epoch 2/50 - Train Loss: 438.9683 - Val Loss: 58.2725 - Train Acc: 0.7115 - Val Acc: 0.6303
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td30_Tw10.pth
Epoch 3/50 - Train Loss: 371.9378 - Val Loss: 47.9647 - Train Acc: 0.7565 - Val Acc: 0.6633
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td30_Tw10.pth
Epoch 4/50 - Train Loss: 348.6307 - Val Loss: 53.7893 - Train Acc: 0.7758 - Val Acc: 0.6628
Epoch 5/50 - Train Loss: 339.3365 - Val Loss: 44.8948 - Train Acc: 0.7848 - Val Acc: 0.6859
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td30_Tw10.pth
Epoch 6/50 - Train Loss: 308.2572 - Val Loss: 46.7206 - Train Acc: 0.7995 - Val Acc: 0.6909
💾 Be

/tmp/ipykernel_2409692/2838713473.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


🔍 Class Weights Tensor:
tensor([2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844,
        2.4844, 2.4844, 2.4844], device='cuda:0')


...

/tmp/ipykernel_2409692/2838713473.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


🔍 Class Weights Tensor:
tensor([2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835,
        2.4835, 2.4835, 2.4835], device='cuda:0')


/home/ea301b/anaconda3/envs/dl_satang/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1/50 - Train Loss: 375.2404 - Val Loss: 38.2006 - Train Acc: 0.4834 - Val Acc: 0.5785
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td60_Tw20.pth
Epoch 2/50 - Train Loss: 189.2355 - Val Loss: 21.9936 - Train Acc: 0.7519 - Val Acc: 0.6604
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td60_Tw20.pth
Epoch 3/50 - Train Loss: 133.2652 - Val Loss: 18.9182 - Train Acc: 0.8371 - Val Acc: 0.7717
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td60_Tw20.pth
Epoch 4/50 - Train Loss: 107.9509 - Val Loss: 17.8551 - Train Acc: 0.8649 - Val Acc: 0.7775
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/teacher/ae_Td60_Tw20.pth
Epoch 5/50 - Train Loss: 99.5399 - Val Loss: 17.3192 - Train Acc: 0.8799 - Val Acc: 0.7635
Epoch 6/50 - Train Loss: 96.6750 - Val Loss: 15.6313 - Train Acc: 0.8797 - Val Acc: 0.8126
💾 Best

/tmp/ipykernel_2409692/2838713473.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


In [66]:
import torch
import torch.nn as nn

class StudentCNN(nn.Module):
    def __init__(self, input_length, num_classes=12):
        super().__init__()
        self.streams = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(1, 4, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv1d(4, 8, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveAvgPool1d(1),
                nn.Flatten()
            ) for _ in range(8)
        ])

        self.fc = nn.Sequential(
            nn.Linear(8 * 8, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

        self.proj = nn.Linear(8 * 8, 128)

    def forward(self, x, return_features=False):
        B, T, C = x.shape
        assert C == 8, f"Expected 8 feature streams, got {C}"

        # Split into 8 streams
        streams = [x[:, :, i].unsqueeze(1) for i in range(C)]  # (B, 1, T)
        features = [self.streams[i](streams[i]) for i in range(8)]  # list of (B, 8)

        x = torch.cat(features, dim=1)  # (B, 64)

        if return_features:
            feat_proj = self.proj(x)  # (B, 128)
            return self.fc(x), None, feat_proj  # dummy recon for consistency

        return self.fc(x)


In [67]:
import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, student_feat, teacher_feat, true_labels, T=3.0, alpha=0.5, beta=0.3, gamma=0.2):
    """
    alpha: weight for hard loss (CE)
    beta: weight for soft loss (KL)
    gamma: weight for feature-based distillation (MSE)
    T: temperature for soft distillation
    """
    # Hard loss
    ce_loss = F.cross_entropy(student_logits, true_labels)

    # Soft loss (logits)
    soft_teacher = F.softmax(teacher_logits / T, dim=1)
    soft_student = F.log_softmax(student_logits / T, dim=1)
    kl_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (T * T)

    # Feature loss (projection-matched MSE)
    feat_loss = F.mse_loss(student_feat, teacher_feat)

    return alpha * ce_loss + beta * kl_loss + gamma * feat_loss


In [68]:
def train_distilled(student, teacher, train_loader, val_loader, device,
                    epochs=50, lr=0.0005, class_weights=None,
                    T=3.0, alpha=0.5, beta=0.3, gamma=0.2,
                    save_path="best_student.pth"):

    teacher.eval()
    student.to(device)
    teacher.to(device)

    optimizer = optim.Adam(student.parameters(), lr=lr)
    ce_criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_val_acc = 0
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        student.train()
        total_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # === Forward passes
            student_logits, _, student_feat = student(inputs, return_features=True)
            with torch.no_grad():
                teacher_logits, _, teacher_feat = teacher(inputs, return_features=True)

            # === Distillation loss
            loss = distillation_loss(
                student_logits, teacher_logits,
                student_feat, teacher_feat,
                labels,
                T=T, alpha=alpha, beta=beta, gamma=gamma
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (student_logits.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)

        # === Validation ===
        student.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logits = student(inputs)  # ✅ works as intended
                loss = ce_criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"[Distill] Epoch {epoch+1:02d}/{epochs} - "
              f"Loss: {total_loss:.4f} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student.state_dict(), save_path)
            print(f"💾 Best student model saved to: {save_path}")

    student.load_state_dict(torch.load(save_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [73]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import os

detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]

base_dir = "/home/HardDisk/Satang/thesis_proj"
teacher_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi", "ae", "teacher")  # AE teacher
student_save_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi", "ae", "student")
os.makedirs(student_save_dir, exist_ok=True)

for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)

        print(f"\n🚀 Distilling AE→StudentCNN for Td={T_d}, Tw={T_w} (T_len={T_len})")

        # === Paths ===
        folder_name = f"X_csv_split_{T_len}"
        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")
        teacher_path = os.path.join(teacher_dir, f"ae_Td{T_d}_Tw{T_w}.pth")
        student_path = os.path.join(student_save_dir, f"student_from_ae_Td{T_d}_Tw{T_w}.pth")

        # === 1. Load Data ===
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. Encode & Balance ===
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)
        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, T_len, NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Dataset & Dataloader ===
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        class_weights = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 4. Load AE Teacher ===
        teacher = AEWithClassifier(
            input_length=T_len,
            feature_dim=NUM_FEATURES,
            latent_dim=64,
            num_classes=len(label_encoder.classes_),
            proj_dim=128
        )
        teacher.load_state_dict(torch.load(teacher_path, map_location=device))
        teacher.to(device)
        teacher.eval()
        for p in teacher.parameters():
            p.requires_grad = False

        # === 5. Init Student CNN & Train ===
        student = StudentCNN(
            input_length=T_len,
            num_classes=len(label_encoder.classes_)
        ).to(device)

        train_accs, val_accs, train_losses, val_losses = train_distilled(
            student=student,
            teacher=teacher,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            lr=0.0001,
            class_weights=class_weights_tensor,
            T=2.5,
            alpha=0.7,
            beta=0.2,
            gamma=0.1,
            save_path=student_path
        )

        # === 6. Final Evaluation ===
        student.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print("\n📊 Final Classification Report (StudentCNN):")
        print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))
        print(f"✅ Student model saved: {student_path}")



🚀 Distilling AE→StudentCNN for Td=30, Tw=10 (T_len=21)


/tmp/ipykernel_2409692/657140011.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load(teacher_path, map_location=device))


[Distill] Epoch 01/50 - Loss: 2650.9437 - Train Acc: 0.2726 - Val Acc: 0.4760
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td30_Tw10.pth
[Distill] Epoch 02/50 - Loss: 1880.5299 - Train Acc: 0.5487 - Val Acc: 0.5286
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td30_Tw10.pth
[Distill] Epoch 03/50 - Loss: 1576.0470 - Train Acc: 0.6191 - Val Acc: 0.5451
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td30_Tw10.pth
[Distill] Epoch 04/50 - Loss: 1403.5020 - Train Acc: 0.6565 - Val Acc: 0.5516
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td30_Tw10.pth
[Distill] Epoch 05/50 - Loss: 1293.6901 - Train Acc: 0.6787 - Val Acc: 0.6202
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cro

/tmp/ipykernel_2409692/342412524.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))



📊 Final Classification Report (StudentCNN):
              precision    recall  f1-score   support

    AESCrypt       0.99      1.00      0.99        85
      Cerber       0.81      0.77      0.79       189
    Darkside       0.82      0.81      0.82       323
       Excel       1.00      1.00      1.00       147
     Firefox       1.00      1.00      1.00       204
   GandCrab4       0.68      0.76      0.71       319
        Ryuk       0.80      0.77      0.79       196
     SDelete       1.00      1.00      1.00        79
  Sodinokibi       0.86      0.71      0.78       205
  TeslaCrypt       0.83      1.00      0.90        85
    WannaCry       1.00      1.00      1.00        79
         Zip       1.00      0.99      0.99        85

    accuracy                           0.86      1996
   macro avg       0.90      0.90      0.90      1996
weighted avg       0.86      0.86      0.86      1996

✅ Student model saved: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/st

...

/tmp/ipykernel_2409692/342412524.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))
/tmp/ipykernel_2409692/657140011.py:57: 

[Distill] Epoch 01/50 - Loss: 1793.8849 - Train Acc: 0.1650 - Val Acc: 0.2519
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw15.pth
[Distill] Epoch 02/50 - Loss: 1550.8307 - Train Acc: 0.4515 - Val Acc: 0.4797
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw15.pth
[Distill] Epoch 03/50 - Loss: 1279.7166 - Train Acc: 0.6733 - Val Acc: 0.6013
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw15.pth
[Distill] Epoch 04/50 - Loss: 1152.6546 - Train Acc: 0.7365 - Val Acc: 0.6127
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw15.pth
[Distill] Epoch 05/50 - Loss: 1069.8414 - Train Acc: 0.7609 - Val Acc: 0.6380
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cro

/tmp/ipykernel_2409692/342412524.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))
/tmp/ipykernel_2409692/657140011.py:57: 

[Distill] Epoch 01/50 - Loss: 2001.8457 - Train Acc: 0.1603 - Val Acc: 0.2494
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw20.pth
[Distill] Epoch 02/50 - Loss: 1716.9503 - Train Acc: 0.4508 - Val Acc: 0.4778
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw20.pth
[Distill] Epoch 03/50 - Loss: 1488.4073 - Train Acc: 0.6379 - Val Acc: 0.5433
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw20.pth
[Distill] Epoch 04/50 - Loss: 1350.2644 - Train Acc: 0.6925 - Val Acc: 0.5890
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/student/student_from_ae_Td60_Tw20.pth
[Distill] Epoch 05/50 - Loss: 1255.0166 - Train Acc: 0.7243 - Val Acc: 0.6112
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cro

/tmp/ipykernel_2409692/342412524.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


In [74]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_student_baseline(student, train_loader, val_loader, device,
                           epochs=50, lr=0.0005, class_weights=None,
                           save_path="best_student_baseline.pth"):

    student.to(device)
    optimizer = optim.Adam(student.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_val_acc = 0
    train_losses, train_accuracies, val_accuracies, val_losses = [], [], [], []

    for epoch in range(epochs):
        student.train()
        total_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            logits = student(inputs)  # 🔹 Only logits expected
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (logits.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)

        # === Validation ===
        student.eval()
        val_correct, val_loss = 0, 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logits = student(inputs)
                loss = criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        val_accuracies.append(val_acc)
        val_losses.append(val_loss)

        print(f"[Baseline] Epoch {epoch+1:02d}/{epochs} - "
              f"Loss: {total_loss:.4f} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student.state_dict(), save_path)
            print(f"💾 Saved best standalone student model to: {save_path}")

    student.load_state_dict(torch.load(save_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [79]:
baseline_save_dir = "/home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student"
os.makedirs(baseline_save_dir, exist_ok=True)

for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)

        print(f"\n🚀 Training Standalone Student for Td={T_d}, Tw={T_w} (T_len={T_len})")

        # === Paths
        folder_name = f"X_csv_split_{T_len}"
        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")
        student_model_path = os.path.join(baseline_save_dir, f"student_baseline_Td{T_d}_Tw{T_w}.pth")

        # === 1. Load Data
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. Encode & Balance
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)

        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)
        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, T_len, NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Dataset & Loader
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === 4. Compute Class Weights
        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 5. Initialize Standalone Student
        student_baseline = StudentCNN(
            input_length=T_len,
            num_classes=len(label_encoder.classes_)
        ).to(device)

        # === 6. Train (No KD)
        train_accs_b, val_accs_b, train_losses_b, val_losses_b = train_student_baseline(
            student=student_baseline,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            lr=0.00005,
            class_weights=class_weights_tensor,
            save_path=student_model_path
        )

        print(f"✅ Saved standalone student model: {student_model_path}")



🚀 Training Standalone Student for Td=30, Tw=10 (T_len=21)
[Baseline] Epoch 01/50 - Loss: 1405.6159 - Train Acc: 0.1477 - Val Acc: 0.2099
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 02/50 - Loss: 1146.1133 - Train Acc: 0.3425 - Val Acc: 0.4163
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 03/50 - Loss: 872.6926 - Train Acc: 0.5178 - Val Acc: 0.4749
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 04/50 - Loss: 755.2142 - Train Acc: 0.5709 - Val Acc: 0.4975
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoc

/tmp/ipykernel_2409692/914780513.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Loss: 1827.1933 - Train Acc: 0.1558 - Val Acc: 0.2434
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 02/50 - Loss: 1354.1988 - Train Acc: 0.4285 - Val Acc: 0.4544
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 03/50 - Loss: 1014.7215 - Train Acc: 0.5843 - Val Acc: 0.5114
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 04/50 - Loss: 866.1570 - Train Acc: 0.6311 - Val Acc: 0.5528
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 05/50 - Loss: 774.4451 - Train Acc: 0.6607 - Val Acc: 0.

/tmp/ipykernel_2409692/914780513.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Loss: 2935.8526 - Train Acc: 0.3755 - Val Acc: 0.5201
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw20.pth
[Baseline] Epoch 02/50 - Loss: 1712.0840 - Train Acc: 0.6523 - Val Acc: 0.5925
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw20.pth
[Baseline] Epoch 03/50 - Loss: 1385.6784 - Train Acc: 0.7106 - Val Acc: 0.6147
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw20.pth
[Baseline] Epoch 04/50 - Loss: 1197.6419 - Train Acc: 0.7406 - Val Acc: 0.6436
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td30_Tw20.pth
[Baseline] Epoch 05/50 - Loss: 1068.0820 - Train Acc: 0.7626 - Val Acc: 

...

/tmp/ipykernel_2409692/914780513.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Loss: 626.1540 - Train Acc: 0.0949 - Val Acc: 0.2165
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 02/50 - Loss: 611.2647 - Train Acc: 0.1352 - Val Acc: 0.2671
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 03/50 - Loss: 581.4301 - Train Acc: 0.2445 - Val Acc: 0.4405
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 04/50 - Loss: 438.2678 - Train Acc: 0.5033 - Val Acc: 0.5089
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 05/50 - Loss: 343.6388 - Train Acc: 0.6115 - Val Acc: 0.539

/tmp/ipykernel_2409692/914780513.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Loss: 683.4314 - Train Acc: 0.1103 - Val Acc: 0.2237
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 02/50 - Loss: 655.4610 - Train Acc: 0.1894 - Val Acc: 0.3665
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 03/50 - Loss: 571.1371 - Train Acc: 0.3454 - Val Acc: 0.4344
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 04/50 - Loss: 420.4691 - Train Acc: 0.5527 - Val Acc: 0.4977
💾 Saved best standalone student model to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 05/50 - Loss: 352.2167 - Train Acc: 0.6357 - Val Acc: 0.565

/tmp/ipykernel_2409692/914780513.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


In [80]:
import os
import torch
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader

# === CONFIG ===
detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]
NUM_FEATURES = 8
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_dir = "/home/HardDisk/Satang/thesis_proj/"
base_teacher_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi", "ae", "teacher")
student_kd_dir = os.path.join(base_dir, "Deep_Learning","cross_archi","ae", "student")
student_baseline_dir = os.path.join(base_dir, "Deep_Learning","cross_archi","ae", "baseline_student")
csv_output_path = os.path.join(base_dir, "Deep_Learning","cross_archi","ae", "results", "model_eval_summary.csv")
os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)

# # === YOUR MODULE IMPORTS ===
# from your_module import load_split_from_folder, MultiStreamDataset, RansomwareTransformer, StudentCNN

results = []

def evaluate_and_log(model, model_path, test_loader, label_encoder, T_d, T_w, model_type, results_list):
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            # Handle models that return a tuple (logits, recon)
            if isinstance(model, AEWithClassifier):
                logits, _ = model(inputs)
            else:
                logits = model(inputs)

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    report = classification_report(all_labels, all_preds, target_names=label_encoder.classes_, output_dict=True)
    acc = accuracy_score(all_labels, all_preds)
    
    results_list.append({
        "Td": T_d,
        "Tw": T_w,
        "T_len": T_d - T_w + 1,
        "Model": model_type,
        "Accuracy": round(acc, 4),
        "Precision_macro": round(report["macro avg"]["precision"], 4),
        "Recall_macro": round(report["macro avg"]["recall"], 4),
        "F1_macro": round(report["macro avg"]["f1-score"], 4),
        "F1_weighted": round(report["weighted avg"]["f1-score"], 4),
    })

# === MAIN EVALUATION LOOP ===
for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)
        folder_name = f"X_csv_split_{T_len}"
        test_path = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name, "test")

        # === Load test set ===
        X_test_raw, y_test_raw = load_split_from_folder(test_path, expected_shape)
        label_encoder = LabelEncoder()
        label_encoder.fit(y_test_raw)

        test_dataset = MultiStreamDataset(X_test_raw, y_test_raw, label_encoder, augment=False)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === Teacher ===
        teacher_path = os.path.join(base_teacher_dir, f"ae_Td{T_d}_Tw{T_w}.pth")
        teacher = AEWithClassifier(
        input_length=T_len,                # ✅ must match original
        feature_dim=NUM_FEATURES,
        latent_dim=64,
        num_classes=len(label_encoder.classes_),
        proj_dim=128
    )
        evaluate_and_log(teacher, teacher_path, test_loader, label_encoder, T_d, T_w, "Teacher", results)

        # === Student with KD ===
        student_kd_path = os.path.join(student_kd_dir, f"student_from_ae_Td{T_d}_Tw{T_w}.pth")
        student_kd = StudentCNN(input_length=T_len, num_classes=len(label_encoder.classes_))
        evaluate_and_log(student_kd, student_kd_path, test_loader, label_encoder, T_d, T_w, "Student_KD", results)

        # === Student Baseline ===
        student_b_path = os.path.join(student_baseline_dir, f"student_baseline_Td{T_d}_Tw{T_w}.pth")
        student_b = StudentCNN(input_length=T_len, num_classes=len(label_encoder.classes_))
        evaluate_and_log(student_b, student_b_path, test_loader, label_encoder, T_d, T_w, "Student_Baseline", results)

# === Save to CSV ===
df_results = pd.DataFrame(results)
df_results.to_csv(csv_output_path, index=False)
print(f"\n✅ All evaluation results saved to {csv_output_path}")


/tmp/ipykernel_2409692/4053020706.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))
/tmp/ipykernel_2409


✅ All evaluation results saved to /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/ae/results/model_eval_summary.csv
